# Objective
In this notebook I am going to try and build a tool that can help me identify the flaws of my accent.
To do this, I'll need some labelled data: (voice recording and accent).

On HuggingFace I found this dataset https://huggingface.co/datasets/westbrook/English_Accent_DataSet which seems to be exactly what we needed!

Now let's see if we can classify some clips!

In [2]:
from datasets import load_dataset

train_dataset = load_dataset("westbrook/English_Accent_DataSet", split="train")
valid_dataset = load_dataset("westbrook/English_Accent_DataSet", split="validation")
test_dataset  = load_dataset("westbrook/English_Accent_DataSet", split="test")

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

In [3]:
import torch as t
import torch.nn as nn
import torchvision.transforms.functional as F

import pandas as pd
import numpy as np
import random

device = 'cuda' if t.cuda.is_available() else 'cpu'
print(f"using device: {device}")

# Reproducibility -> Also ensures same train / validation / test split every time
t.manual_seed(42)
random.seed(42)
np.random.seed(42)

t.cuda.empty_cache()

using device: cuda


In [4]:
train_dataset = train_dataset.with_format('torch')
valid_dataset = valid_dataset.with_format('torch')
test_dataset = test_dataset.with_format('torch')

print(train_dataset.column_names)
print(train_dataset[0])

['audio_id', 'audio', 'raw_text', 'gender', 'speaker_id', 'duration', 'accent', 'split']
{'audio_id': 'edacc_EDACC-C06_1', 'audio': {'path': 'edacc_EDACC-C06_1.wav', 'array': tensor([-6.1035e-03, -8.7585e-03, -7.2937e-03,  ..., -1.5259e-04,
        -9.1553e-05, -9.1553e-05]), 'sampling_rate': tensor(16000)}, 'raw_text': 'OKAY NOW FOR A REGULAR CONVERSATION SO UH WOULD YOU RATHER GO TO THE BEACH TODAY OR DO CLUB GOLF DISCUSS', 'gender': 'male', 'speaker_id': 'EDACC-C06-A', 'duration': tensor(8.3500), 'accent': tensor(15), 'split': 'training'}


In [5]:
from encodec import EncodecModel
from encodec.utils import convert_audio

import numpy as np
from IPython.display import Audio, display

with t.no_grad():
    # Instantiate a pretrained EnCodec model
    encodec = EncodecModel.encodec_model_24khz()
    encodec.set_target_bandwidth(6.0)

    # Load and pre-process the audio waveform
    wav, sr = train_dataset[0]['audio']['array'], train_dataset[0]['audio']['sampling_rate'].item()

    display(Audio(wav.squeeze(), rate=sr))

    wav = wav.unsqueeze(0).unsqueeze(0)
    wav = convert_audio(wav, sr, encodec.sample_rate, encodec.channels)

    display(Audio(wav.squeeze(), rate=encodec.sample_rate))

    # Extract discrete codes from EnCodec
    encoded_frames = encodec.encode(wav)
    codes = t.cat([encoded[0] for encoded in encoded_frames], dim=-1) 

    de = encodec.decode(encoded_frames)

    display(Audio(de.squeeze(), rate=encodec.sample_rate))

/home/andreacacioli/miniconda3/envs/experiments/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
